In [22]:
import pandas as pd
from torchaudio import load
# !pip install torch

In [23]:
# !pip install pandas
# !pip install open_clip_torch
import torch
print(torch.__version__)
torch.cuda.is_available()
import numpy as np
import transformers
import librosa
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from transformers import ClapModel, ClapProcessor, ClapConfig, AutoFeatureExtractor
import os 
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

2.1.2+cu121
cuda:0


In [24]:
model = ClapModel.from_pretrained("laion/larger_clap_general")
processor = ClapProcessor.from_pretrained("laion/larger_clap_general")

In [25]:
model.to(device)
import pandas as pd

In [26]:
df = pd.read_csv('final_v2_dataframe.csv')

In [27]:
df['file_path'] = df['file_path'].str.replace('./Dataset/', './')

In [28]:
df

,track_name,file_path,sentiment
0,20657,./Common/020657_0s.mp3,1
1,20657,./Common/020657_10s.mp3,1
2,20657,./Common/020657_20s.mp3,1
3,20939,./Common/020939_0s.mp3,1
4,20939,./Common/020939_10s.mp3,1
...,...,...,...
4248,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0
4249,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0
4250,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0
4251,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0


In [29]:
df['sentiment'].value_counts()

sentiment
1    2173
0    2080
Name: count, dtype: int64

In [30]:
def getAudioFeature(audio):
    waveform, sample_rate = load(audio)
    inputs = processor(audios=waveform.numpy(), return_tensors="pt", sampling_rate=48000).to(device)
    audio_embed = model.get_audio_features(**inputs)[0]
    return audio_embed.detach().cpu().numpy()

In [31]:
df['material_id'] = "audio" + df.index.astype(str)

In [32]:
df

,track_name,file_path,sentiment,material_id
0,20657,./Common/020657_0s.mp3,1,audio0
1,20657,./Common/020657_10s.mp3,1,audio1
2,20657,./Common/020657_20s.mp3,1,audio2
3,20939,./Common/020939_0s.mp3,1,audio3
4,20939,./Common/020939_10s.mp3,1,audio4
...,...,...,...,...
4248,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0,audio4248
4249,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0,audio4249
4250,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0,audio4250
4251,You Will Never See Me Coming - NEFFEX,./Common/You Will Never See Me Coming - NEFFEX...,0,audio4251


In [21]:
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        audio = row['file_path'] 
        text_feature = getAudioFeature(audio)
        material_id = row['material_id']
        torch.save(text_feature, f'./audio_embeddings/{material_id}.pt')
    except Exception as e:
        
        # Handle exceptions here if needed
        print(f"Error processing row {index}: {e}")

100%|███████████████████████████████████████████████████████████████████████████████| 4253/4253 [18:38<00:00,  3.80it/s]
